# Detecting Motion with OpenCV
---
This motion detecting model can be divided into 3 parts.
1. Compare previous frame and current frame to get differences on the frame.
2. Find out contours from the frame that has differences.
3. Draw bounding box around each contour on the frame.

In [14]:
import cv2
import sys
import numpy as np

### Load a video file

In [24]:
# Create a video capture object
video = cv2.VideoCapture("people.mp4")

# Check that video is opened
if not video.isOpened():
    print("Could not open video")
    sys.exit()

# Read video with read() method
ok, frame = video.read()
if not ok:
    print("Cannot read video file")
    sys.exit()

### Detect object's motion from the video and draw bounding box around it.

In [25]:
# Define previous_frame to None
previous_frame = None

while True:

    # Get frame from video
    ok, frame = video.read()
    if not ok:
        break

    # Convert frame to RGB frame
    rgb_frame = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2RGB)
    # rgb_frame_2 will be used for contour window
    rgb_frame_2 = cv2.cvtColor(src=frame, code=cv2.COLOR_BGR2RGB)

    # Convert the image to grayscale format
    gray_frame = cv2.cvtColor(rgb_frame, cv2.COLOR_BGR2GRAY)
    # Blur the image for smoothing
    prepared_frame = cv2.GaussianBlur(src=gray_frame, ksize=(5, 5), sigmaX=0)

    # Set previous frame and continue if there is None
    if (previous_frame is None):
        # First frame; there is no previous one yet
        previous_frame = prepared_frame
        continue


    ### Part 1 ###
    # Calculate difference and update previous frame
    diff_frame = cv2.absdiff(src1=previous_frame, src2=prepared_frame)
    previous_frame = prepared_frame

    # Dilute the image a bit to make differences more seeable; more suitable for contour detection
    kernel = np.ones((5, 5))
    diff_frame = cv2.dilate(diff_frame, kernel, 1)

    # Only take different areas that are different enough (>20 / 255)
    thresh_frame = cv2.threshold(src=diff_frame, thresh=20, maxval=255, type=cv2.THRESH_BINARY)[1]

    # Display thresh_frame on the window
    cv2.imshow('1. Show changes on the frame', thresh_frame)


    ### Part 2 ###
    # Find contours from thresh_frame
    contours, _ = cv2.findContours(image=thresh_frame, mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_SIMPLE)
    
    # Drawing contours on rgb_frame_2
    cv2.drawContours(image=rgb_frame_2, contours=contours, contourIdx=-1, color=(0, 255, 0), thickness=2, lineType=cv2.LINE_AA)
    
    # Display rgb_frame_2 on the window
    cv2.imshow('2. Contours', rgb_frame_2)


    ### Part 3 ###
    # Drawing rectangles on rgb_frame
    for contour in contours:
        if cv2.contourArea(contour) < 700:
            # too small: skip!
            continue
        (x, y, w, h) = cv2.boundingRect(contour)
        cv2.rectangle(img=rgb_frame, pt1=(x, y), pt2=(x + w, y + h), color=(0, 255, 0), thickness=2)
    
    # Display rgb_frame on the window
    cv2.imshow('Motion detector', rgb_frame)

    if (cv2.waitKey(50) == 27):
        # out.release()
        break

# Cleanup
cv2.destroyAllWindows()